In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import os
import csv
import sklearn
import matplotlib.image as mpimg
import keras 

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
!unzip ./drive/My\ Drive/data.zip -d .

In [ ]:
# Create generator for dataset with images and steering measurements

# path where the data is stored 
path = './data' 

# read in the csv lines 
samples = []
with open(path+'/driving_log.csv') as file:
    reader = csv.reader(file)
    for line in reader: 
        samples.append(line)

# split csv lines into test, validation, and test
from sklearn.model_selection import train_test_split
train_samples, test_samples = train_test_split(samples, test_size=.2)
test_samples, valid_samples = train_test_split(samples, test_size=.25)


In [ ]:
def generator(samples, batch_size=32):
    while True: 
        sklearn.utils.shuffle(samples)
        # for each batch
        for start in range(0, len(samples), batch_size//2): # batch size is divided by 2 because we add the flipped images as well
            batch_samples = samples[start:start+batch_size]

            images = []
            angles = []

            # for each csv line
            for batch_sample in batch_samples:
                
                # get image path
                name = path+'/IMG/'+batch_sample[0].split('\\')[-1]
                
                # add image and angle
                center_image = mpimg.imread(name)
                center_angle=float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                
                # flip image 
                center_image_flip = np.fliplr(center_image)
                center_angle_flip = -center_angle
                images.append(center_image_flip)
                angles.append(center_angle_flip)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [ ]:
# Build model

# original shape of the image
input_shape = (160,320,3)

# first input layer of the model
car_input = keras.layers.Input(shape=input_shape)

# crop image to remove unnecessary 
cropped_input = keras.layers.Cropping2D(cropping=((50, 25), (0, 0)))(car_input)

crop_input_shape = (85, 300, 3) # new input shape (85, 300, 3)

# normalize image data
normal_input = keras.layers.Lambda(lambda x: (x/255.0)-.5)(cropped_input)

# resize to fit keras inception model
resize_shape = 128
resized_input = keras.layers.Lambda(lambda image: tf.image.resize_images( 
    image, (resize_shape, resize_shape)))(normal_input)


In [ ]:
conv1 = keras.layers.Conv2D(16, (5,5), activation='relu')(resized_input)
pool1 = keras.layers.MaxPooling2D()(conv1)

conv2 = keras.layers.Conv2D(32, (5,5), activation='relu')(pool1)
pool2 = keras.layers.MaxPooling2D()(conv2)

conv3 = keras.layers.Conv2D(64, (5,5), activation='relu')(pool2)
pool3 = keras.layers.MaxPooling2D()(conv3)

In [ ]:
# flatten and add more fully connected layers to get steering measurement prediction
flat = keras.layers.Flatten()(pool3)

dropout1 = keras.layers.Dropout(.5)(flat)

fc1 = keras.layers.Dense(1028, activation=tf.nn.relu)(dropout1)

dropout2 = keras.layers.Dropout(.5)(fc1)

fc2 = keras.layers.Dense(256, activation=tf.nn.relu)(dropout2)

prediction = keras.layers.Dense(1)(fc2)


In [ ]:
from keras.models import Model

# creates the model
model = Model(inputs=car_input, outputs=prediction)

# compile the model
model.compile(optimizer='Adam', loss='mse')

# check the summary of model to confirm architecture
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_3 (Cropping2D)    (None, 85, 320, 3)        0         
_________________________________________________________________
lambda_5 (Lambda)            (None, 85, 320, 3)        0         
_________________________________________________________________
lambda_6 (Lambda)            (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 124, 124, 16)      1216      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 58, 58, 32)        1283

In [ ]:
batch_size = 32
from math import ceil

train_generator = generator(train_samples, batch_size=batch_size)
valid_generator = generator(valid_samples, batch_size=batch_size)

In [ ]:
history = model.fit_generator(train_generator, 
            steps_per_epoch=ceil(len(train_samples)/batch_size), 
            validation_data=valid_generator, 
            validation_steps=ceil(len(valid_samples)/batch_size), 
            epochs=5, verbose=1)

model.save('model.h5')

Epoch 1/5
275/275 [==============================] - 25s 92ms/step - loss: 0.0200 - val_loss: 0.0083
Epoch 2/5
275/275 [==============================] - 23s 85ms/step - loss: 0.0186 - val_loss: 0.0101
Epoch 3/5
275/275 [==============================] - 23s 84ms/step - loss: 0.0174 - val_loss: 0.0068
Epoch 4/5
275/275 [==============================] - 23s 85ms/step - loss: 0.0170 - val_loss: 0.0078
Epoch 5/5
275/275 [==============================] - 23s 85ms/step - loss: 0.0156 - val_loss: 0.0051
